# Week 1 Homework

## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

> [!NOTE]
> You can run Elastic Search 8.4.3 using the docker command [here](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/README.md#16-searching-with-elasticsearch)
>
> Or via [docker compose](https://docs.docker.com/compose/gettingstarted) by downloading the following [compose](./docker-compose.yml) file I have provided and running `docker compose up` in the same directory as the YAML file

### Notes:

:white_check_mark: '42f05b9372a9a4a470db3b52817899b99a76ee73'

In [1]:
import requests

response = requests.get("http://localhost:9200")
data = response.json()

data["version"]["build_hash"]

'42f05b9372a9a4a470db3b52817899b99a76ee73'

## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [2]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

Note that you need to have the `requests` library:

```bash
pip install requests
```

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

### Notes:

:white_check_mark: `index`

In [3]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

response_create = es_client.indices.create(index=index_name, body=index_settings)
print(response_create)

for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'}


  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

Now let's search in our index. 

For a query "How do I execute a command in a running docker container?", what's the score for the top ranking result? 

Use only `question` and `text` fields and give `question` a boost of 4

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

### Notes:

:white_check_mark: 84.05

In [4]:
query = "How do I execute a command in a running docker container?"

search_query_q3 = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
        }
    }
}

response_q3 = es_client.search(index=index_name, body=search_query_q3)

top_score = response_q3["hits"]["hits"][0]["_score"]
print(f"Q3 Answer: The `_score` field value is {top_score}")

Q3 Answer: The `_score` field value is 84.050095


## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

### Notes:

:white_check_mark: How do I copy files from a different folder into docker container’s working directory?

In [5]:
search_query_q4 = search_query_q3.copy()
search_query_q4["size"] = 3
search_query_q4["query"]["bool"]["filter"] = {
    "term": {"course": "machine-learning-zoomcamp"}
}

response_q4 = es_client.search(index=index_name, body=search_query_q4)

result_docs = []

for hit in response_q4["hits"]["hits"]:
    result_docs.append(hit["_source"])

print("The 3 result questions:")
for i, doc in enumerate(result_docs):
    print(f"{i + 1}. {doc['question']}")
print()

third_question = result_docs[2]["question"]
print(
    f"""Q4 Answer: The 3rd question returned by the search engine is
'{third_question}'
""".strip()
)

The 3 result questions:
1. How do I debug a docker container?
2. How do I copy files from my local machine to docker container?
3. How do I copy files from a different folder into docker container’s working directory?

Q4 Answer: The 3rd question returned by the search engine is
'How do I copy files from a different folder into docker container’s working directory?'


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. Let's use these templates:

In [6]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

Separate context entries by two linebreaks (`\n\n`)

Put the data from Q4 there. 

What's the length of the result? (use the `len` function)

* 962
* 1462
* 1962
* 2462

### Notes:

:white_check_mark: 1462

In [7]:
def build_prompt(query: str, search_results: list):
    context = ""

    for doc in search_results:
        context = (
            context
            + context_template.format(question=doc["question"], text=doc["text"])
            + "\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


prompt = build_prompt(query, result_docs)
print(f"Q5 Answer: The length of the result is {len(prompt)}.")

Q5 Answer: The length of the result is 1462.


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

### Notes:

:white_check_mark: 322

In [8]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

num_tokens = len(encoding.encode(prompt))
print(f"Q6 Answer: Our prompt has {num_tokens} tokens.")

Q6 Answer: Our prompt has 322 tokens.


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.


> [!NOTE]
> Run the following commands in your terminal to pull the Ollama `Phi 3` model
> ```bash
> # Use this command if you prefer to run Ollama using a docker container
> docker exec -it ollama bash
> 
> # Pulls the Phi 3 Mini model
> ollama pull phi3
> ```

In [9]:
from openai import OpenAI


def llm(client: "OpenAI", prompt: str):
    response = client.chat.completions.create(
        model="phi3",
        messages=[
            {"role": "user", "content": prompt},
        ],
    )

    return response.choices[0].message.content


client = OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama",
)

answer = llm(client, prompt)

print(
    f"""Query:
{query}

Response:
{answer}
""".strip()
)

Query:
How do I execute a command in a running docker container?

Response:
 To execute a command in a running docker container, you can use the `docker exec` command followed by the specific container ID and your desired command. Here's how to do it:

1. If the container is already running, execute a command in the specific container using the following format:
   ```
   docker exec -it <container-id> <command>
   ```

For example, if you want to run "ls" within a running container with ID `abc123`, you would use:
   ```
   docker exec -it abc123 ls
   ```


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [10]:
input_token_cost = 0.005 / 1_000
output_token_cost = 0.015 / 1_000

request_count = 1_000
request_cost = (150 * input_token_cost) + (250 * output_token_cost)
total_cost = request_count * request_cost
print("It will cost ${:.2f} to run {:,} requests.".format(total_cost, request_count))

It will cost $4.50 to run 1,000 requests.
